# TCC - Predição de notas capes de programas de pós-graduação
#### Discente: Rodrigo Augusto Valeretto - NUSP: 10684792
#### Orientador: Adenilso da Silva Simão

### Imports

In [1]:
import pandas as pd
import numpy as np
import pymongo
import chardet
import matplotlib.pyplot as plt

from sklearn import tree

from sklearn.tree import (
    DecisionTreeClassifier
)
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier
)
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    ConfusionMatrixDisplay,
    RocCurveDisplay
)
from sklearn.model_selection import (
    cross_val_score,
    GridSearchCV,
    train_test_split
)

### Definindo variáveis globais que mudam a estrutura do documento

In [2]:
train_size = 0.3
test_size = 0.7
random_state = 42
plt.rcParams['figure.figsize'] = (16, 9)

### Definição de funções

In [3]:
def separate_xy(df, data_cols, res_col):
    df_copy = df.copy(deep=True)
    x = df_copy[data_cols]
    y = df_copy[res_col]
    
    return x, y

In [4]:
def detect_encoding(nome_arquivo):
    with open(nome_arquivo, "rb") as rawdata:
        result = chardet.detect(rawdata.read(10000))
    print(result)

In [5]:
def instanciar_collections(sucupira_db):
    progs_c = sucupira_db["Programas"]
    anais_c = sucupira_db["Producoes_anais"]
    artpe_c = sucupira_db["Producoes_artpe"]
    docentes_c = sucupira_db["Docentes"]
    btd_c = sucupira_db["Teses_Dissertacoes"]
    
    return progs_c, anais_c, artpe_c, docentes_c, btd_c

In [6]:
def instanciar_dataframes(sucupira_db):
    print("Instanciando collections e dataframes para", sucupira_db.name)
    progs_c, anais_c, artpe_c, docentes_c, btd_c = instanciar_collections(sucupira_db)

    progs_df = pd.DataFrame(progs_c.aggregate([{ "$project": { "_id": 0 } }]))
    
    print("Finalizado")    
    return progs_df

In [7]:
def separa_grandes_areas_mongo(sucupira_db):
    progs_c, anais_c, artpe_c, docentes_c, btd_c = instanciar_collections(sucupira_db)
    
    areas_conhecimento_list = progs_c.distinct("NM_GRANDE_AREA_CONHECIMENTO")
    
    dfs_por_area = {}
    
    for area in areas_conhecimento_list:
        df = pd.DataFrame(progs_c.find({
            "NM_GRANDE_AREA_CONHECIMENTO": { "$eq": area }
        }))
        
        dfs_por_area[area] = df
    
    return dfs_por_area

In [8]:
def separa_grandes_areas_df(df):
    areas_conhecimento_list = sorted(df["NM_GRANDE_AREA_CONHECIMENTO"].unique())
    df_copy = df.copy(deep=True)
    
    dfs_por_area = {}
    
    for area in areas_conhecimento_list:
        df = df_copy[df_copy["NM_GRANDE_AREA_CONHECIMENTO"] == area].reset_index(drop=True)
        
        dfs_por_area[area] = df
    
    return dfs_por_area

In [9]:
def shared_columns(array01, array02):
    same_columns = []
    for column in array01:
        if(column in array02):
            same_columns.append(column)
    return same_columns

In [10]:
def shared_columns_inf(*arrays):
    result = arrays[0]
    for array in arrays:
        result = shared_columns(result, array)
    return result

In [11]:
def instancia_tree(classifier="Decision", criterion="gini", x_train=None, y_train=None, plot_tree=False):
    global count
    has_tree = ["Decision", "Balanced"]
    
    match classifier:
        case "Decision":
            dt = DecisionTreeClassifier(criterion=criterion, random_state=random_state)
        case "Bagging":
            base_estimator = DecisionTreeClassifier(criterion=criterion, random_state=random_state)
            dt = BaggingClassifier(base_estimator=base_estimator, random_state=random_state)
        case "Random":
            dt = RandomForestClassifier(criterion=criterion, random_state=random_state)
        case "Balanced":
            dt = DecisionTreeClassifier(criterion=criterion, random_state=random_state, class_weight="balanced")
        case "RandomBalanced":
            dt = RandomForestClassifier(criterion=criterion, random_state=random_state, class_weight="balanced")
        case _:
            dt = DecisionTreeClassifier(criterion=criterion, random_state=random_state)
            
    if(x_train is not None and y_train is not None):
        dt.fit(x_train, y_train)
    
        if(plot_tree and classifier in has_tree):
            fig = plt.figure()
            tree.plot_tree(dt, feature_names=data_cols, filled=True)
            fig.savefig("./figs/" + classifier + str(count) + ".eps", format="eps", dpi=2400)
            count += 1
            
    return dt

In [12]:
def score_func(y_true, y_pred, y_prob=None, dt=None, metric="accuracy", display_matrix=False, display_roc=False):
    if(display_matrix):
        ConfusionMatrixDisplay.from_predictions(y_true, y_pred, cmap="tab20b") 
        
    if(display_roc):
        RocCurveDisplay.from_predictions(y_true, y_pred, pos_label=7)
    
    unique_y_true = np.unique(y_true)
    
    labels = unique_y_true if len(unique_y_true) > len(dt.classes_) else dt.classes_
    
    match metric:
        case "roc_auc":
            if(y_prob is None):
                raise "Erro, y_prob é parâmetro obrigatório para essa métrica."
            return roc_auc_score(y_true, y_prob, multi_class="ovo", labels=labels)
        case "precision":
            return precision_score(y_true, y_pred, average="macro")
        case "recall":
            return recall_score(y_true, y_pred, average="macro")
        case _:
            return accuracy_score(y_true, y_pred)

In [13]:
def aplica_previsao(df, df_test=None, classifier="Decision", criterion="gini", metric="accuracy", plot_tree=False, display_matrix=False, display_roc=False):
    if(df_test is None):
        x, y = separate_xy(df, data_cols, res_col)

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, test_size=test_size, random_state=random_state)
    else:
        x_train, y_train = separate_xy(df, data_cols, res_col)
        x_test, y_test = separate_xy(df_test, data_cols, res_col)
    
    dt = instancia_tree(classifier, criterion, x_train, y_train, plot_tree)
    
    y_pred = dt.predict(x_test)
    y_prob = dt.predict_proba(x_test)
    diff = y_pred - y_test
    score = score_func(y_test, y_pred, y_prob, dt, metric, display_matrix, display_roc)
    
    print("Score:", score)
    
    return score, diff

In [14]:
def segundo_teste(df, df_test=None, classifier="Decision", criterion="gini", metric="accuracy", plot_tree=False, display_matrix=False, display_roc=False):
    scores = []
    diffs = []
    # Treinando com a base toda e aplicando a arvore em vários dataframes separados
    # por grande área para verificar se o score aumenta
    if(df_test is None):
        df_train, df_test = train_test_split(df, train_size=train_size, test_size=test_size, random_state=random_state)
        x_train, y_train = separate_xy(df_train, data_cols, res_col)
    else:
        x_train, y_train = separate_xy(df, data_cols, res_col)
    
    dt = instancia_tree(classifier, criterion, x_train, y_train, plot_tree)
   
    dfs_grandes_areas = separa_grandes_areas_df(df_test)

    for area in dfs_grandes_areas.keys():
        df_ga_test_x, df_ga_test_y = separate_xy(dfs_grandes_areas[area], data_cols, res_col)
        y_predicted = dt.predict(df_ga_test_x)
        y_prob = dt.predict_proba(df_ga_test_x)
        score = score_func(df_ga_test_y, y_predicted, y_prob, dt, metric, display_matrix, display_roc)
        diff = y_predicted - df_ga_test_y
        scores.append(score)
        diffs.append(diff)
        print("Resultado para área:", area)
        print("Score:", score)
        print()
    
    return scores, diffs

In [15]:
def terceiro_teste(df, df_test=None, classifier="Decision", criterion="gini", metric="accuracy", plot_tree=False, display_matrix=False, display_roc=False):
    scores = []
    diffs = []
    # Treinando por grande área e testando por grande área
    dfs_grandes_areas = separa_grandes_areas_df(df)
    dfs_grandes_areas_test = (None if df_test is None else separa_grandes_areas_df(df_test))
    for area in dfs_grandes_areas.keys():
        print("Resultado para área:", area)
        df_area_test = (None if dfs_grandes_areas_test is None else dfs_grandes_areas_test[area])
        score, diff = aplica_previsao(dfs_grandes_areas[area], df_area_test, classifier, criterion, metric, plot_tree, display_matrix, display_roc)
        scores.append(score)
        diffs.append(diff)
        print()
    
    return scores, diffs 

In [16]:
def realiza_testes(df, df_test=None, classifier="Decision", criterion="gini", metric="accuracy", plot_tree=False, display_matrix=False, display_roc=False):
    # Primeiro teste
    # Treinando com a base toda e aplicando na base toda
    print("**PRIMEIRO TESTE**")
    scores_1, diff_1 = aplica_previsao(df, df_test, classifier, criterion, metric, plot_tree, display_matrix, display_roc)
    print()

    # Segundo teste
    print("**SEGUNDO TESTE**")
    scores_2, diff_2 = segundo_teste(df, df_test, classifier, criterion, metric, plot_tree, display_matrix, display_roc)
    print()

    # Terceiro teste
    print("**TERCEIRO TESTE**")
    scores_3, diff_3 = terceiro_teste(df, df_test, classifier, criterion, metric, plot_tree, display_matrix, display_roc)
    print()
    
    return (scores_1, scores_2, scores_3), (diff_1, diff_2, diff_3)

In [17]:
def preenche_df_scores(scores, indexes, labels, it, df=None):
    if(df is None):
        index_list = sorted(indexes*3, key=indexes.index)
        df = pd.DataFrame(index=index_list, columns=labels)
        
    df.iloc[it*3, 0] = scores[0]
    df.iloc[it*3 + 1, 1:] = scores[1]
    df.iloc[it*3 + 2, 1:] = scores[2]
    return df

In [18]:
def preenche_df_diffs(diffs, indexes, labels, it, df=None):
    if(df is None):
        index_list = sorted(indexes*3, key=indexes.index)
        df = pd.DataFrame(index=index_list, columns=labels)
        
    df.iloc[it*3, 0] = np.sum(diffs[0])
    df.iloc[it*3 + 1, 1:] = [np.sum(diffs[1][i]/abs(diffs[1][i])) for i in range(0, len(diffs[1]))]
    df.iloc[it*3 + 2, 1:] = [np.sum(diffs[2][i]/abs(diffs[2][i])) for i in range(0, len(diffs[2]))]
    return df

### Instanciando dataframes

In [19]:
client = pymongo.MongoClient()
sucupira_db_2013 = client["Sucupira_2013"]
sucupira_db_2017 = client["Sucupira_2017"]

In [20]:
progs_df_2017 = instanciar_dataframes(sucupira_db_2017)

Instanciando collections e dataframes para Sucupira_2017
Finalizado


In [21]:
progs_df_2017

,QTD_DISCENTES_DOUTORADO,QTD_DISCENTES_MESTRADO,QTD_DISCENTES,QTD_DOCENTES_DOUTORADO,QTD_DOCENTES_MESTRADO,QTD_DOCENTES,CD_PROGRAMA_IES,QTD_ARTPE,QTD_ANAIS,AN_BASE,...,NM_PROGRAMA_IES,NM_PROGRAMA_IDIOMA,NM_GRAU_PROGRAMA,CD_CONCEITO_PROGRAMA,AN_INICIO_PROGRAMA,AN_INICIO_CURSO,DS_SITUACAO_PROGRAMA,DT_SITUACAO_PROGRAMA,DURACAO_PROGRAMA,FORMANDOS_POR_DOCENTE
0,17,60,77,17,0,17,10001018002P1,172,157,2020,...,BIOLOGIA EXPERIMENTAL,EXPERIMENTAL BIOLOGY,MESTRADO/DOUTORADO,4,2001,2001/2005,EM FUNCIONAMENTO,30DEC2013:00:00:00,12,4.529412
1,0,0,0,13,0,13,10001018041P7,90,25,2020,...,CONSERVAÇÃO E USO DE RECURSOS NATURAIS,CONSERVATION AND USE OF NATURAL RESOURCES,MESTRADO,5,2019,2019,EM FUNCIONAMENTO,28AUG2019:00:00:00,0,0.000000
2,0,56,56,16,0,16,10001018009P6,126,84,2020,...,PSICOLOGIA,GRADUATE PROGRAM PSYCOLOGY,MESTRADO,3,2009,2009,EM FUNCIONAMENTO,29MAY2013:00:00:00,4,3.500000
3,0,87,87,16,0,16,10001018010P4,128,55,2020,...,LETRAS,MASTER DEGREE: MESTRADO ACADÊMICO EM LETRAS,MESTRADO,3,2010,2010,EM FUNCIONAMENTO,23APR2013:00:00:00,3,5.437500
4,0,0,72,17,0,17,10001018016P2,258,142,2020,...,EDUCAÇÃO ESCOLAR,PROFESSIONAL MASTERS OF SCHOOL EDUCATION,MESTRADO PROFISSIONAL/DOUTORADO PROFISSIONAL,4,2014,2014,EM FUNCIONAMENTO,10APR2013:00:00:00,0,4.235294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774,0,32,32,44,0,44,28022017002P1,113,198,2018,...,MICROBIOLOGIA AGRÍCOLA,AGRICULTURAL MICROBIOLOGY,MESTRADO,3,2008,2008,EM FUNCIONAMENTO,11FEB2014:00:00:00,6,0.727273
4775,0,32,32,44,0,44,28022017002P1,113,198,2017,...,MICROBIOLOGIA AGRÍCOLA,AGRICULTURAL MICROBIOLOGY,MESTRADO,3,2008,2008,EM FUNCIONAMENTO,11FEB2014:00:00:00,6,0.727273
4776,0,23,23,38,0,38,24009016029P7,51,66,2019,...,EXPLORAÇÃO PETROLÍFERA E MINERAL,PETROLEUM AND MINERAL EXPLORATION,MESTRADO,3,2013,2013,EM FUNCIONAMENTO,18SEP2013:00:00:00,0,0.605263
4777,0,23,23,38,0,38,24009016029P7,51,66,2018,...,EXPLORAÇÃO PETROLÍFERA E MINERAL,PETROLEUM AND MINERAL EXPLORATION,MESTRADO,3,2013,2013,EM FUNCIONAMENTO,18SEP2013:00:00:00,0,0.605263


In [22]:
progs_df_2017[progs_df_2017["DURACAO_PROGRAMA"] <= 0]

,QTD_DISCENTES_DOUTORADO,QTD_DISCENTES_MESTRADO,QTD_DISCENTES,QTD_DOCENTES_DOUTORADO,QTD_DOCENTES_MESTRADO,QTD_DOCENTES,CD_PROGRAMA_IES,QTD_ARTPE,QTD_ANAIS,AN_BASE,...,NM_PROGRAMA_IES,NM_PROGRAMA_IDIOMA,NM_GRAU_PROGRAMA,CD_CONCEITO_PROGRAMA,AN_INICIO_PROGRAMA,AN_INICIO_CURSO,DS_SITUACAO_PROGRAMA,DT_SITUACAO_PROGRAMA,DURACAO_PROGRAMA,FORMANDOS_POR_DOCENTE
1,0,0,0,13,0,13,10001018041P7,90,25,2020,...,CONSERVAÇÃO E USO DE RECURSOS NATURAIS,CONSERVATION AND USE OF NATURAL RESOURCES,MESTRADO,5,2019,2019,EM FUNCIONAMENTO,28AUG2019:00:00:00,0,0.000000
4,0,0,72,17,0,17,10001018016P2,258,142,2020,...,EDUCAÇÃO ESCOLAR,PROFESSIONAL MASTERS OF SCHOOL EDUCATION,MESTRADO PROFISSIONAL/DOUTORADO PROFISSIONAL,4,2014,2014,EM FUNCIONAMENTO,10APR2013:00:00:00,0,4.235294
5,0,0,0,12,0,12,10001018043P0,63,7,2020,...,FILOSOFIA,PHILOSOPHY,MESTRADO,5,2019,2019,EM FUNCIONAMENTO,17SEP2019:00:00:00,0,0.000000
8,0,0,42,16,0,16,10001018039P2,207,117,2020,...,DIREITOS HUMANOS E DESENVOLVIMENTO DA JUSTIÇA,PROFESSIONAL INTERDISCIPLINARY MASTER ON HUMAN...,MESTRADO PROFISSIONAL,3,2017,2017,EM FUNCIONAMENTO,02MAY2017:00:00:00,0,2.625000
9,0,0,0,19,0,19,10001018044P6,49,4,2020,...,AGROECOSSISTEMAS AMAZÔNICOS,AMAZONIANS AGROECOSYSTEMS,MESTRADO,5,2020,2020,EM FUNCIONAMENTO,24MAR2020:00:00:00,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,0,0,0,39,0,39,33149011003P8,77,28,2017,...,ADMINISTRAÇÃO DAS MICRO E PEQUENAS EMPRESAS,DOCTORAL IN SMALL BUSINESS MANAGEMENT,DOUTORADO,4,2017,2017,EM FUNCIONAMENTO,15MAR2017:00:00:00,0,0.000000
4771,0,0,0,9,0,10,31106005001P8,7,36,2017,...,DESENVOLVIMENTO REGIONAL E MEIO AMBIENTE,REGIONAL DEVELOPMENT AND THE ENVIRONMENT,MESTRADO PROFISSIONAL,3,2014,2014,EM FUNCIONAMENTO,04AUG2014:00:00:00,0,0.000000
4776,0,23,23,38,0,38,24009016029P7,51,66,2019,...,EXPLORAÇÃO PETROLÍFERA E MINERAL,PETROLEUM AND MINERAL EXPLORATION,MESTRADO,3,2013,2013,EM FUNCIONAMENTO,18SEP2013:00:00:00,0,0.605263
4777,0,23,23,38,0,38,24009016029P7,51,66,2018,...,EXPLORAÇÃO PETROLÍFERA E MINERAL,PETROLEUM AND MINERAL EXPLORATION,MESTRADO,3,2013,2013,EM FUNCIONAMENTO,18SEP2013:00:00:00,0,0.605263


In [23]:
progs_df_2013 = instanciar_dataframes(sucupira_db_2013)

Instanciando collections e dataframes para Sucupira_2013
Finalizado


In [24]:
progs_df_2013

,CD_PROGRAMA_IES,QTD_DISCENTES_DOUTORADO,QTD_DISCENTES_MESTRADO,QTD_DISCENTES,QTD_DOCENTES_DOUTORADO,QTD_DOCENTES_MESTRADO,QTD_DOCENTES,QTD_ARTPE,QTD_ANAIS,AN_BASE,...,CD_CONCEITO_PROGRAMA,AN_INICIO_CURSO,IN_REDE,SG_ENTIDADE_ENSINO_REDE,DS_SITUACAO_PROGRAMA,DT_SITUACAO_PROGRAMA,DS_CLIENTELA_QUADRIENAL_2017,AN_INICIO_PROGRAMA,DURACAO_PROGRAMA,FORMANDOS_POR_DOCENTE
0,10001018002P1,20,52,72,20,0,20,178,157,2016,...,4,2001/2005,NÃO,NaN,EM FUNCIONAMENTO,30DEC2013:00:00:00,AVALIAÇÃO,2001,12,3.600000
1,10001018004P4,1,55,56,29,0,29,259,254,2016,...,4,1999/2013,NÃO,NaN,EM FUNCIONAMENTO,13NOV2013:00:00:00,AVALIAÇÃO,1999,14,1.931034
2,10001018005P0,0,54,54,16,0,16,154,282,2016,...,4,2006/2015,NÃO,NaN,EM FUNCIONAMENTO,19JUN2013:00:00:00,AVALIAÇÃO,2006,7,3.375000
3,10001018006P7,0,65,65,12,0,12,130,325,2016,...,3,2006,NÃO,NaN,EM FUNCIONAMENTO,30DEC2013:00:00:00,AVALIAÇÃO,2006,7,5.416667
4,10001018009P6,0,67,67,13,0,13,67,143,2016,...,3,2009,NÃO,NaN,EM FUNCIONAMENTO,29MAY2013:00:00:00,AVALIAÇÃO,2009,4,5.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4089,32010010010P5,1,6,7,81,0,81,274,181,2015,...,3,2013/2013,SIM,UFU,EM FUNCIONAMENTO,30OCT2012:00:00:00,AVALIAÇÃO,2013,0,0.086420
4090,32010010010P5,1,6,7,81,0,81,274,181,2014,...,3,2013/2013,SIM,UFU,EM FUNCIONAMENTO,30OCT2012:00:00:00,AVALIAÇÃO,2013,0,0.086420
4091,32010010010P5,1,6,7,81,0,81,274,181,2013,...,3,2013/2013,SIM,UFU,EM FUNCIONAMENTO,30OCT2012:00:00:00,AVALIAÇÃO,2013,0,0.086420
4092,25001019063P2,0,0,10,13,0,15,63,21,2013,...,4,2001,NÃO,NaN,EM FUNCIONAMENTO,16DEC2013:00:00:00,AVALIAÇÃO,2001,12,0.666667


### Aplicando algoritmos de previsão

In [25]:
data_cols = ["QTD_DOCENTES", "QTD_DOCENTES_DOUTORADO", "QTD_DOCENTES_MESTRADO", "QTD_DISCENTES", "QTD_DISCENTES_DOUTORADO", "QTD_DISCENTES_MESTRADO", "QTD_ARTPE", "QTD_ANAIS", "DURACAO_PROGRAMA", "FORMANDOS_POR_DOCENTE"]
res_col = "CD_CONCEITO_PROGRAMA"
indexes = ["Decision", "Bagging", "Random", "Balanced", "RandomBalanced"]
labels = sorted(progs_df_2013["NM_GRANDE_AREA_CONHECIMENTO"].unique())
df_labels = ["Tudo"] + labels
count = 0
df_test = None
metric = "roc_auc"
criterion = "entropy"
plot_tree = False
display_matrix = False
display_roc = False

print("Realizando testes para df 2013\n")
scores_2013, diffs_2013 = realiza_testes(progs_df_2013, df_test=df_test, classifier="Decision", criterion=criterion, metric=metric, plot_tree=plot_tree, display_matrix=display_matrix, display_roc=display_roc)
print()

df_scores = preenche_df_scores(scores_2013, indexes, df_labels, 0)
df_diffs = preenche_df_diffs(diffs_2013, indexes, df_labels, 0)

Realizando testes para df 2013

**PRIMEIRO TESTE**
Score: 0.6223928882329449

**SEGUNDO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.5830262637541582

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.6038512684124386

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.6266905112907686

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.6281808035714286

Resultado para área: CIÊNCIAS HUMANAS
Score: 0.6150718778645573

Resultado para área: CIÊNCIAS SOCIAIS APLICADAS
Score: 0.644903729663088

Resultado para área: ENGENHARIAS
Score: 0.6312945271278605

Resultado para área: LINGÜÍSTICA, LETRAS E ARTES
Score: 0.5995334470636889

Resultado para área: MULTIDISCIPLINAR
Score: 0.6150570566225705


**TERCEIRO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.6334057208380026

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.6115811169002658

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.6232792536522236

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.656109926107

Visualizando os dados percebemos que boa parte dos algoritmos estima um valor maior do que é de fato a nota Capes do programa. Além disso percebemos que apesar do teste 3 acertar menos que o teste 2, ele provê estimativas mais próximas quando erra.

In [26]:
print("Realizando testes para df 2013\n")
scores_2013, diffs_2013 = realiza_testes(progs_df_2013, df_test=df_test, classifier="Bagging", criterion=criterion, metric=metric, plot_tree=plot_tree, display_matrix=display_matrix, display_roc=display_roc)
print()

df_scores = preenche_df_scores(scores_2013, indexes, df_labels, 1, df_scores)
df_diffs = preenche_df_diffs(diffs_2013, indexes, df_labels, 1, df_diffs)

Realizando testes para df 2013

**PRIMEIRO TESTE**
Score: 0.7804642483054025

**SEGUNDO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.8065687012133429

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.8077157020973511

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.7676634641011972

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.7340597098214285

Resultado para área: CIÊNCIAS HUMANAS
Score: 0.7730413676393257

Resultado para área: CIÊNCIAS SOCIAIS APLICADAS
Score: 0.7466048283055541

Resultado para área: ENGENHARIAS
Score: 0.7605000140675522

Resultado para área: LINGÜÍSTICA, LETRAS E ARTES
Score: 0.8107852133305762

Resultado para área: MULTIDISCIPLINAR
Score: 0.7765319611382964


**TERCEIRO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.7793421262135707

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.7223190710618177

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.7642680900071135

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.78297503076

In [27]:
print("Realizando testes para df 2013\n")
scores_2013, diffs_2013 = realiza_testes(progs_df_2013, df_test=df_test, classifier="Random", criterion=criterion, metric=metric, plot_tree=plot_tree, display_matrix=display_matrix, display_roc=display_roc)
print()

df_scores = preenche_df_scores(scores_2013, indexes, df_labels, 2, df_scores)
df_diffs = preenche_df_diffs(diffs_2013, indexes, df_labels, 2, df_diffs)

Realizando testes para df 2013

**PRIMEIRO TESTE**
Score: 0.8080820678069545

**SEGUNDO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.8194059514917384

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.8343270083196945

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.7974923998464095

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.8013257998511906

Resultado para área: CIÊNCIAS HUMANAS
Score: 0.8104679958162926

Resultado para área: CIÊNCIAS SOCIAIS APLICADAS
Score: 0.7621317672014769

Resultado para área: ENGENHARIAS
Score: 0.7805494141823117

Resultado para área: LINGÜÍSTICA, LETRAS E ARTES
Score: 0.8157845563654537

Resultado para área: MULTIDISCIPLINAR
Score: 0.7802663629141431


**TERCEIRO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.8165076625788495

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.7853769979253539

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.8018567202964457

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.79601910392

In [28]:
print("Realizando testes para df 2013\n")
scores_2013, diffs_2013 = realiza_testes(progs_df_2013, df_test=df_test, classifier="Balanced", criterion=criterion, metric=metric, plot_tree=plot_tree, display_matrix=display_matrix, display_roc=display_roc)
print()

df_scores = preenche_df_scores(scores_2013, indexes, df_labels, 3, df_scores)
df_diffs = preenche_df_diffs(diffs_2013, indexes, df_labels, 3, df_diffs)

Realizando testes para df 2013

**PRIMEIRO TESTE**
Score: 0.6209718463867624

**SEGUNDO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.6382620036290041

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.5971659619930897

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.6260048328622434

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.6164806547619046

Resultado para área: CIÊNCIAS HUMANAS
Score: 0.6186435031027206

Resultado para área: CIÊNCIAS SOCIAIS APLICADAS
Score: 0.5600267824866757

Resultado para área: ENGENHARIAS
Score: 0.6284853723579215

Resultado para área: LINGÜÍSTICA, LETRAS E ARTES
Score: 0.6191034687758478

Resultado para área: MULTIDISCIPLINAR
Score: 0.6130230310372479


**TERCEIRO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.6486130891616317

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.652115088285301

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.6193549253036164

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.673488167726

In [29]:
print("Realizando testes para df 2013\n")
scores_2013, diffs_2013 = realiza_testes(progs_df_2013, df_test=df_test, classifier="RandomBalanced", criterion=criterion, metric=metric, plot_tree=plot_tree, display_matrix=display_matrix, display_roc=display_roc)
print()

df_scores = preenche_df_scores(scores_2013, indexes, df_labels, 4, df_scores)
df_diffs = preenche_df_diffs(diffs_2013, indexes, df_labels, 4, df_diffs)

Realizando testes para df 2013

**PRIMEIRO TESTE**
Score: 0.8034707913735314

**SEGUNDO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.8141783561418805

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.8282281721676668

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.7937462476006459

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.7935285295758929

Resultado para área: CIÊNCIAS HUMANAS
Score: 0.8021444346935542

Resultado para área: CIÊNCIAS SOCIAIS APLICADAS
Score: 0.7504896550694871

Resultado para área: ENGENHARIAS
Score: 0.7877277223954785

Resultado para área: LINGÜÍSTICA, LETRAS E ARTES
Score: 0.8125333329025365

Resultado para área: MULTIDISCIPLINAR
Score: 0.7874196241717859


**TERCEIRO TESTE**
Resultado para área: CIÊNCIAS AGRÁRIAS
Score: 0.8090222243189651

Resultado para área: CIÊNCIAS BIOLÓGICAS
Score: 0.7926843850683309

Resultado para área: CIÊNCIAS DA SAÚDE
Score: 0.7981863197684962

Resultado para área: CIÊNCIAS EXATAS E DA TERRA
Score: 0.79695756780

O score representa a porcentagem de acerto que nosso modelo tem com relação à amostra real dos dados. <br>
Agora seria necessário melhorar o score, para isso podemos pensar em novas variáveis que podem ser adicionadas que contribuam positivamente para a arvore de decisão, testar diferentes classificadores ou algoritmos de regressão para ver como se comportam com a mesma amostra ou também testar configurações diferentes no mesmo classificador para analisar se a solução melhora ou piora. 

Uma ideia interessante para melhorar a performance do nosso algoritmo é pegar o melhor deles e aplicar pesos personalizados de acordo com as variáveis que fazem mais sentido para nós, caso tenhamos uma ideia do que impacta mais ou não na nota capes.

In [30]:
df_scores

,Tudo,CIÊNCIAS AGRÁRIAS,CIÊNCIAS BIOLÓGICAS,CIÊNCIAS DA SAÚDE,CIÊNCIAS EXATAS E DA TERRA,CIÊNCIAS HUMANAS,CIÊNCIAS SOCIAIS APLICADAS,ENGENHARIAS,"LINGÜÍSTICA, LETRAS E ARTES",MULTIDISCIPLINAR
Decision,0.622393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Decision,NaN,0.583026,0.603851,0.626691,0.628181,0.615072,0.644904,0.631295,0.599533,0.615057
Decision,NaN,0.633406,0.611581,0.623279,0.65611,0.671429,0.614631,0.664157,0.673807,0.591115
Bagging,0.780464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bagging,NaN,0.806569,0.807716,0.767663,0.73406,0.773041,0.746605,0.7605,0.810785,0.776532
Bagging,NaN,0.779342,0.722319,0.764268,0.782975,0.794083,0.717522,0.752284,0.821576,0.769412
Random,0.808082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Random,NaN,0.819406,0.834327,0.797492,0.801326,0.810468,0.762132,0.780549,0.815785,0.780266
Random,NaN,0.816508,0.785377,0.801857,0.796019,0.831298,0.762917,0.794438,0.82933,0.796325
Balanced,0.620972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_diffs

,Tudo,CIÊNCIAS AGRÁRIAS,CIÊNCIAS BIOLÓGICAS,CIÊNCIAS DA SAÚDE,CIÊNCIAS EXATAS E DA TERRA,CIÊNCIAS HUMANAS,CIÊNCIAS SOCIAIS APLICADAS,ENGENHARIAS,"LINGÜÍSTICA, LETRAS E ARTES",MULTIDISCIPLINAR
Decision,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Decision,NaN,46.0,10.0,74.0,-13.0,-11.0,-45.0,17.0,-4.0,-3.0
Decision,NaN,18.0,-5.0,-32.0,-9.0,3.0,-7.0,9.0,-15.0,14.0
Bagging,-178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bagging,NaN,26.0,0.0,42.0,-38.0,-55.0,-71.0,18.0,-22.0,-72.0
Bagging,NaN,7.0,-25.0,-16.0,-31.0,-26.0,-45.0,-13.0,-3.0,-32.0
Random,-144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Random,NaN,29.0,3.0,41.0,-24.0,-55.0,-66.0,7.0,-24.0,-63.0
Random,NaN,27.0,-13.0,-38.0,-24.0,-21.0,-49.0,-15.0,2.0,-14.0
Balanced,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Gerando resultado final para dataset de 2017 a 2020

In [75]:
def calcula_previsao_final(df_train, df_test, classifier, criterion, plot_tree=False):
    x_train, y_train = separate_xy(df_train, data_cols, res_col)
    x_test, y_test = separate_xy(df_test, data_cols, res_col)
    
    dt = instancia_tree(classifier, criterion, x_train, y_train, plot_tree)
    
    y_pred = dt.predict(x_test)
    y_prob = dt.predict_proba(x_test)
    
    df_res = pd.concat([df_test["CD_PROGRAMA_IES"], x_test, pd.DataFrame(y_pred, columns=["CONCEITO_PREVISTO"])], axis=1)
    
    return df_res

In [76]:
df_res = calcula_previsao_final(progs_df_2013, progs_df_2017, classifier="Random", criterion="entropy", plot_tree=False)

In [78]:
df_res[["CD_PROGRAMA_IES", "CONCEITO_PREVISTO"]].to_csv("conceitos_previstos.csv", index=False)